# Summary

Computes various summary statistics describing the collection of observations.  In particular note the list showing the last observation from each telescope; this is the easiest way to check whether new observations are included yet.

In [6]:
%load_ext autoreload

%autoreload 2

import os
from glob import glob
import subprocess
import shutil
import traceback
import random
import pickle
from os.path import join

import joblib
from concurrent.futures import ProcessPoolExecutor
import re

from backports import tempfile

import matplotlib
matplotlib.rcParams['savefig.dpi'] = 144
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma

import psrchive

import pipe

plt.viridis()

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Overall summary

In [12]:
n_obs = {}
t_total = {}
tbw_total = {}
mjd_start = {}
mjd_end = {}

for d in sorted(glob(join("data","obs","*"))):
    M = pickle.load(open(join(d,"meta.pickle"),"rb"))
    k = M["tel"], M["band"]
    n_obs[k] = n_obs.setdefault(k,0) + 1
    t_total[k] = t_total.setdefault(k,0) + M["length"]
    mjd_start[k] = min(mjd_start.setdefault(k,1e10),M["mjd"])
    mjd_end[k] = max(mjd_start.setdefault(k,0),M["mjd"])
    #tbw_total[k] = 

In [13]:
print("{tel:5s}{band:>5s}{n:>5s}{t:>10s}{m:>14s}   {r:^12s}".format(
        tel="tel", band="band", n="n", t="T (h)", m="mean T (min)",
        r="MJD range"))
print("-"*54)
for k in sorted(n_obs.keys()):
    tel, band = k
    n = n_obs[k]
    t = t_total[k]/3600
    m = t/n*60
    s, e = mjd_start[k], mjd_end[k]
    print("{tel:5s}{band:5d}{n:5d}{t:10.1f}{m:14.1f}   {s:5.0f}--{e:5.0f}".format(**locals()))

tel   band    n     T (h)  mean T (min)    MJD range  
------------------------------------------------------
AO     350    3       0.9          17.7   57059--57432
AO     430   36      12.9          21.5   56071--57833
AO    1400   92      58.9          38.4   55990--57833
AO    2000    7       3.2          27.0   56769--57552
AO    3000    1       0.7          40.3   56782--56782
GBT   1400  172     236.0          82.3   55962--57888
WSRT   350   20      17.3          52.0   55960--56485
WSRT  1400  439     836.7         114.4   55957--56496


## Longest observations

This is a quick proxy for SNR, which isn't computed until we have a template.

In [14]:
longest = {}
for d in sorted(glob(join("data","obs","*"))):
    M = pickle.load(open(join(d,"meta.pickle"),"rb"))
    k = M["tel"], M["band"]
    if (k not in longest
        or longest[k]["length"]<M["length"]):
        longest[k] = M


In [15]:
for k in longest:
    l = longest[k]
    print k, l["nsubint"], l["length"], l["obs_dir"]

('AO', 2000) 63 4576.99288437 /psr_archive/hessels/archibald/0337+17/obs/56768.79_AO_2000
('AO', 350) 168 1796.86460998 /psr_archive/hessels/archibald/0337+17/obs/57058.98_AO_350
('GBT', 1400) 69 42646.8976767 /psr_archive/hessels/archibald/0337+17/obs/56321.01_GBT_1400
('WSRT', 1400) 3239 32389.792704 /psr_archive/hessels/archibald/0337+17/obs/56262.88_WSRT_1400
('AO', 1400) 35 8877.52792533 /psr_archive/hessels/archibald/0337+17/obs/56025.79_AO_1400
('AO', 3000) 44 2416.49165356 /psr_archive/hessels/archibald/0337+17/obs/56781.71_AO_3000
('AO', 430) 261 2791.53515592 /psr_archive/hessels/archibald/0337+17/obs/56070.64_AO_430
('WSRT', 350) 1799 17988.84864 /psr_archive/hessels/archibald/0337+17/obs/56479.20_WSRT_350


## List of runs

See `bulk-processing` for a more detailed summary of processed data.

In [66]:
processing_runs = set()
toa_runs = set()
for d in sorted(glob(join("data","obs","*"))):
    for sd in os.listdir(d):
        if not os.path.isdir(join(d,sd)):
            continue
        processing_runs.add(sd)
        for ssd in os.listdir(join(d,sd)):
            if not os.path.isdir(join(d,sd,ssd)):
                continue
            toa_runs.add((sd,ssd))

In [67]:
sorted(processing_runs)

['toa-testing', 'wideband']

In [68]:
sorted(toa_runs)

[]

## SNRs

This looks at the processed data to find a list of the observations with the best SNR for each mode. The output is intentionally in python-usable format, so that it can be pasted into notebooks that want to work with the few best observations in each mode.

In [16]:
snrs = {}
processing_name = "tuned"
toa_name = "mueller"
for d in sorted(glob(join("data","obs","*",processing_name,toa_name))):
    M = pickle.load(open(join(d,"..","process.pickle"),"rb"))
    M2 = pickle.load(open(join(d,"summary.pickle"),"rb"))
    k = M["tel"], M["band"]
    if k not in snrs:
        snrs[k] = []
    snrs[k].append((M2['snr_sum'],M['name']))
for k in snrs:
    snrs[k].sort()
    snrs[k] = snrs[k][::-1]
    print k, snrs[k][:20]

('WSRT', 350) [(251.18026218134258, '56102.39_WSRT_350'), (245.23569445045879, '56479.20_WSRT_350'), (104.75167204664039, '56485.26_WSRT_350'), (100.63210377531576, '55967.78_WSRT_350'), (80.232637720029473, '56045.74_WSRT_350'), (69.956378173759788, '56004.67_WSRT_350'), (58.591225643360254, '55977.83_WSRT_350'), (54.103708520745428, '55968.77_WSRT_350'), (53.804596846188119, '55972.74_WSRT_350'), (48.37354189264633, '56039.69_WSRT_350'), (32.71782200838949, '55982.90_WSRT_350'), (29.377213229614213, '55966.84_WSRT_350'), (29.282446431742695, '55987.70_WSRT_350'), (28.666824813701474, '56168.95_WSRT_350'), (27.841890534209007, '56000.84_WSRT_350'), (24.363223473499868, '55990.63_WSRT_350'), (23.967162099712926, '55994.76_WSRT_350'), (23.488575414046206, '55961.87_WSRT_350'), (20.915352306050821, '55959.72_WSRT_350'), (16.783416545569992, '56067.54_WSRT_350')]
('AO', 430) [(709.81840051858285, '56070.64_AO_430'), (508.39091994908512, '57628.41_AO_430'), (461.02895391364893, '57808.92_A